# Soil Flow Optimization Notebook

## 1. Importing Packages

In [ ]:
# Import all required libraries
import pyomo.environ as pyo
from pyomo.opt import ProblemFormat
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import time
import json
import multiprocessing as mp
import pickle
import os
from pathlib import Path
from matplotlib.dates import MonthLocator, DateFormatter
import calendar
import ipywidgets as widgets
import gurobipy as gp

# Set up any global configurations
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Defining Functions For Main Script

### 2.1 Loading Project Data

In [ ]:
def load_data():
    try:
        with open('../Data/soil_transport_data.pkl', 'rb') as file:
            data_objects = pickle.load(file)

        return data_objects
    except FileNotFoundError:
        raise FileNotFoundError("Data file 'soil_transport_data.pkl' not found")
    
    

### 2.2 Preprocessing Raw Project Data

In [ ]:
def process_data(data_objects):

    # Extract individual objects from data dictionary
    df_aanvullen = data_objects.get('df_aanvullen', pd.DataFrame())
    df_ontgraven = data_objects.get('df_ontgraven', pd.DataFrame())
    #df_compatibility = data_objects.get('df_compatibility', pd.DataFrame())
    compatibility_dict_civ = data_objects.get('compatibility_dict_civ', {})
    compatibility_dict_env = data_objects.get('compatibility_dict_env', {})
    df_costs = data_objects.get('df_costs', pd.DataFrame())
    df_planning = data_objects.get('df_planning', pd.DataFrame())
    
    required_columns = ['Locatie', 'Grondsoort', 'Volume', 'Civieltechnisch', 'Objecttype']
    for df, name in [(df_aanvullen, 'df_aanvullen'), (df_ontgraven, 'df_ontgraven')]:
        for col in required_columns:
            if col not in df.columns:
                raise ValueError(f"Required column '{col}' not found in {name}")

    # 1. Process supply and demand data
    supply_data = _extract_supply_data(df_ontgraven)
    demand_data = _extract_demand_data(df_aanvullen)
    
    # Combine into layers dataframe
    layers_df = pd.DataFrame(supply_data + demand_data)
    
    # Extract unique cut and fill pits
    cut_pits = layers_df[layers_df['pit_type'] == 'cut']['pit_id'].unique().tolist()
    fill_pits = layers_df[layers_df['pit_type'] == 'fill']['pit_id'].unique().tolist()
    
    # 2. Process costs data
    costs_df = _process_costs_data(df_costs, cut_pits, fill_pits)
    
    # 3. Generate coordinates for visualization
    pit_coords = _generate_pit_coordinates(cut_pits, fill_pits)
    
    # 4. Build soil compatibility matrix combining both civil and environmental constraints
    soil_compatibility = create_soil_compatibility_matrix(
        layers_df, compatibility_dict_civ, compatibility_dict_env
    )
    
    return layers_df, costs_df, pit_coords, soil_compatibility, cut_pits, fill_pits, df_planning


def _extract_supply_data(df_ontgraven):
    supply_data = []
    for _, row in df_ontgraven.iterrows():
        # Create simplified row
        supply_data.append({
            'object': str(row['Object']),
            'pit_id': str(row['Locatie']),
            'soil_type': row['Civieltechnisch'],
            'volume': abs(row['Volume']),
            'pit_type': 'cut',
            'applicability': row['Civieltechnisch'],
            'objecttype': row['Objecttype'],
            'bank_side': row.get('WB/LB', None),
            'env_class': row.get('Milieuklasse', None)
        })
    return supply_data

def _extract_demand_data(df_aanvullen):
    demand_data = []
    for _, row in df_aanvullen.iterrows():
        # Create a standardized soil type string
        soil_type = str(row['Grondsoort'])
        
        # Create simplified row
        demand_data.append({
            'object': str(row['Object']),
            'pit_id': str(row['Locatie']),
            'soil_type': row['Civieltechnisch'],
            'volume': abs(row['Volume']),
            'pit_type': 'fill',
            'applicability': row['Civieltechnisch'],
            'objecttype': row['Objecttype'],
            'bank_side': row.get('WB/LB', None),
            'env_class': row.get('Milieuklasse', None)
        })
    return demand_data

def _process_costs_data(df_costs, cut_pits, fill_pits):
    if 'Locatie' in df_costs.columns:
        # Convert from matrix format to source/destination format
        costs = []
        for _, row in df_costs.iterrows():
            source = row['Locatie']
            for col in df_costs.columns:
                if col != 'Locatie' and col in fill_pits and source in cut_pits:
                    cost_value = row[col]
                    if cost_value > 0:  # Skip zero or negative costs
                        costs.append({
                            'source': source,
                            'destination': col, 
                            'cost': round(float(cost_value), ndigits= 2) 
                        })
        
        costs_df = pd.DataFrame(costs)
    else:
        # Already in source/destination format or empty
        costs_df = df_costs.copy()
    
    # If costs_df is empty, create a default cost structure based on Euclidean distance
    if costs_df.empty:
        print("No costs data found.")

    return costs_df

def _generate_pit_coordinates(cut_pits, fill_pits):
    
    pit_coords = {}
    grid_size = int(np.ceil(np.sqrt(len(cut_pits) + len(fill_pits))))
    
    # Position nodes in a grid
    for i, pit in enumerate(cut_pits):
        x = (i % grid_size) * 10
        y = (i // grid_size) * 10
        pit_coords[pit] = (x, y)
    
    for i, pit in enumerate(fill_pits):
        x = (i % grid_size) * 10
        y = ((i // grid_size) + len(cut_pits) // grid_size + 1) * 10 # Offset y for fill pits so they don't overlap with cut pits
        pit_coords[pit] = (x, y)
    
    # Add storage, disposal and borrow sites
    max_x = max([coords[0] for coords in pit_coords.values()]) + 10 if pit_coords else 100
    max_y = max([coords[1] for coords in pit_coords.values()]) + 10 if pit_coords else 100
    min_x = min([coords[0] for coords in pit_coords.values()]) - 10 if pit_coords else -10
    min_y = min([coords[1] for coords in pit_coords.values()]) - 10 if pit_coords else -10
    
    pit_coords['disposal'] = (min_x, min_y)
    pit_coords['borrow'] = (max_x, max_y)
    pit_coords['storage'] = (max_x * 0.5, min_y * 0.5)
    
    return pit_coords

def create_soil_compatibility_matrix(layers_df, compatibility_dict_civ, compatibility_dict_env=None):
    soil_compatibility = {}
    
    # Extract unique soil details from layers
    cut_layers = layers_df[layers_df['pit_type'] == 'cut']
    fill_layers = layers_df[layers_df['pit_type'] == 'fill']
    
    # For each unique source soil
    for _, cut_row in cut_layers.iterrows():
        source_location = cut_row['objecttype']  # 'Dijk'/'Rivier'
        source_type = cut_row['applicability']   # soil type like 'teelaarde'
        
        # Create a composite key that includes both location and soil type
        source_key = (source_location, source_type)
        
        # Check if this composite key is already in our dictionary
        if source_key not in soil_compatibility:
            soil_compatibility[source_key] = {}
        
        # For each unique receiving soil
        for _, fill_row in fill_layers.iterrows():
            dest_location = fill_row['objecttype']  # destination location
            dest_type = fill_row['applicability']   # destination soil type
            
            # Create a composite key for the destination
            dest_key = (dest_location, dest_type)
            
            # Check civil engineering compatibility
            is_civ_compatible = False
            priority = 0
            
            try:
                if source_key in compatibility_dict_civ:
                    if dest_key in compatibility_dict_civ[source_key]:
                        is_civ_compatible = True
                        priority = compatibility_dict_civ[source_key][dest_key]
            except Exception as e:
                print(f"Error checking civil compatibility: {e}")

            # Environmental compatibility is not required for this project, but code is left for future use
            
            is_env_compatible = True  # Default to True if no env check needed

            # if is_civ_compatible and compatibility_dict_env:
            #     # Get environmental data
            #     source_bank = cut_row.get('bank_side')
            #     source_env = cut_row.get('env_class')
            #     dest_bank = fill_row.get('bank_side')
            #     dest_env = fill_row.get('env_class')
                
            #     # Only check env compatibility if we have the required data
            #     if None not in [source_bank, source_env]:
            #         env_source = (source_location, source_bank, source_env)
            #         env_dest = (dest_location, dest_bank, dest_env)
                    
            #         # This matches the original environmental compatibility check
            #         try:
            #             is_env_compatible = False
            #             if env_source in compatibility_dict_env:
            #                 is_env_compatible = env_dest in compatibility_dict_env[env_source]
            #         except Exception as e:
            #             print(f"Error checking environmental compatibility: {e}")
            #             is_env_compatible = False
            
            # Only add to compatibility matrix if both checks pass
        
            if is_civ_compatible and is_env_compatible:
                soil_compatibility[source_key][dest_key] = priority

    return soil_compatibility

def process_time_windows(df_planning, time_periods, all_objects):

    object_time_windows = {}
    object_end_periods = {}  # Track when each object must be completed
    month_labels = {}  # Store month labels for visualization
    
    try:
        # Convert dates to timestamps
        df_planning['Uitvoering_Start'] = pd.to_datetime(df_planning['Uitvoering_Start'])
        df_planning['Uitvoering_End'] = pd.to_datetime(df_planning['Uitvoering_End'])
        
        # Find min date to determine project start
        min_date = df_planning['Uitvoering_Start'].min().replace(day=1)  # Start at beginning of month
        
        # Store month labels
        for period in range(1, time_periods + 1): # Time periods are 1-based, and determines the total number of months, user-input parameter
            month_date = min_date + pd.DateOffset(months=period-1)
            month_labels[period] = month_date.strftime('%b %Y')
        
        # Map date to a month-based period (1-based)
        def map_date_to_period(date):
            months_diff = (date.year - min_date.year) * 12 + date.month - min_date.month
            period = months_diff + 1  # 1-based indexing
            return min(max(1, period), time_periods)  # Ensure the month lies within total project timespan
        
        # Process each object
        for object_id in all_objects:
            # Find matching row in planning data
            matching_rows = df_planning[df_planning['Object'] == object_id]
            
            if not matching_rows.empty:
                # Get the planning info for this object
                row = matching_rows.iloc[0]
                start_period = map_date_to_period(row['Uitvoering_Start'])
                end_period = map_date_to_period(row['Uitvoering_End'])
                
                # Ensure end_period is not before start_period
                end_period = max(start_period, end_period)
                
                # Assign allowed time periods
                object_time_windows[object_id] = list(range(start_period, end_period + 1))
                object_end_periods[object_id] = end_period
            else:
                print(f"No planning data found for object {object_id}. Using default time periods.")
                # If no match found, allow all time periods
                object_time_windows[object_id] = list(range(1, time_periods + 1))
                object_end_periods[object_id] = time_periods
        
        print(f"Mapped dates to {time_periods} monthly time periods starting from {min_date.strftime('%B %Y')}")
    
    except Exception as e:
        print(f"Error processing time windows: {e}")
        # Fall back to allowing all time periods for all objects
        for object_id in all_objects:
            object_time_windows[object_id] = list(range(1, time_periods + 1))
            object_end_periods[object_id] = time_periods
        
        # Create default month labels
        for period in range(1, time_periods + 1):
            month_labels[period] = f"Month {period}"

    print("Allowed time periods for each object:")
    for obj, periods in object_time_windows.items():
        print(f"  {obj}: {periods}")
    
    return object_time_windows, object_end_periods, month_labels


### 2.3 Creating and Solving Model 

In [ ]:
def create_and_solve_model(layers_df, costs_df, soil_compatibility, cut_pits, fill_pits, pit_coords,
                        time_periods=72, time_limit=300, df_planning=None):
    
    print("\n===== CREATING AND SOLVING MODEL WITH ENHANCED CONSTRAINTS =====")
    
    # Create base model
    model = pyo.ConcreteModel(name="soil_transport")
    
    # Add sets, parameters, variables 
    _initialize_model_sets(model, layers_df, cut_pits, fill_pits, df_planning, time_periods)
    
    _initialize_model_parameters(model, layers_df, costs_df, soil_compatibility)
    
    _setup_allowed_movements(model, soil_compatibility)
    
    _initialize_model_variables(model, time_periods)
    
    # Add constraints
    _add_volume_balance_constraints(model)
    _add_linking_constraints(model)
    _add_time_window_constraints(model)
    _add_capacity_constraints(model)
    _add_inventory_constraints(model)
    _add_priority_constraints(model)
    _add_logic_constraints(model)
    add_integrated_cut_fill_constraints(model)

    # Add objective function
    _create_objective_function(model)

    # Solve model
    print(f"Solving model with {len(model.ALLOWED_MOVEMENTS)} possible soil movements...")
    print(f"Monthly capacity: {pyo.value(model.MONTHLY_CAPACITY)} m³")

    model.write(
        'infeasibility.lp', # Save the model to a file for debugging
        io_options={'symbolic_solver_labels': True}, # Include labels for easier debugging
        format= ProblemFormat.cpxlp  # Use the correct format for Gurobi
    )

    # Solver parameters, adjusted after testing various configurations to optimize performance

    solver = pyo.SolverFactory("gurobi")
    solver.options["IISMethod"] = 1
    solver.options["Threads"] = 7                 # Adjust to all available cores to maximize CPU usage
    solver.options["Method"] = 3                  
    solver.options["Presolve"] = 2               
    solver.options["MIPFocus"] = 1                
    solver.options["Cuts"] = 2                  
    solver.options["GomoryPasses"] = 3          
    solver.options["Heuristics"] = 0.8           
    solver.options["MIPGap"] = 0.2               # Accept solutions within 20% of optimal
    solver.options["TimeLimit"] = 11000            # 3-hour time limit
    solver.options["NodefileStart"] = 0.5       
    solver.options["NodeMethod"] = 1             
    solver.options["MIPSepCuts"] = 2              
    solver.options["ResultFile"] = "best_found_solution.sol"  # Save the solution to a file

    try:
        # Start timing
        start_time = time.time()
        results = solver.solve(model, tee=True)
        solve_time = time.time() - start_time
        
        print(f"Solving completed in {solve_time:.2f} seconds")
        
        if results.solver.status == pyo.SolverStatus.ok:
            current_obj = pyo.value(model.objective)
            print(f"Objective value: {current_obj:.2f}")
            
            return model
        else:
            print("Solver failed to find a solution")
            return None
    except Exception as e:
        print(f"Error solving model: {e}")
        return None



### 2.4 Initializing Sets, Parameters, Variables and the Objective Function

In [ ]:
def _initialize_model_sets(model, layers_df, cut_pits, fill_pits, df_planning, time_periods):
   
    # Sets
    model.T = pyo.Set(initialize=range(1, time_periods + 1), doc="Time periods (months)")
    model.CUT_PITS = pyo.Set(initialize=cut_pits, doc="Cut pits")
    model.FILL_PITS = pyo.Set(initialize=fill_pits, doc="Fill pits")
    model.DISPOSAL_PITS = pyo.Set(initialize=['disposal'], doc="Disposal sites")
    model.BORROW_PITS = pyo.Set(initialize=['borrow'], doc="Borrow sites")
    model.STORAGE_SITES = pyo.Set(initialize=['storage'], doc="Storage sink sites") # Only a single storage site for now, adjust to include more 

    # Get soil types for each pit
    cut_soils = {}
    for pit in cut_pits:
        pit_soils = layers_df[(layers_df['pit_id'] == pit) & (layers_df['pit_type'] == 'cut')]['soil_type'].unique()
        cut_soils[pit] = list(pit_soils)
    
    fill_soils = {}
    for pit in fill_pits:
        pit_soils = layers_df[(layers_df['pit_id'] == pit) & (layers_df['pit_type'] == 'fill')]['soil_type'].unique()
        fill_soils[pit] = list(pit_soils)
    
    # Define cut/fill layers sets by soil type
    model.CUT_LAYERS = pyo.Set(
        initialize=[(i, s) for i in cut_pits for s in cut_soils.get(i, [])],
        doc="Cut pit soils (pit, soil_type)",
        dimen=2
    )
    
    model.FILL_LAYERS = pyo.Set(
        initialize=[(j, s) for j in fill_pits for s in fill_soils.get(j, [])],
        doc="Fill pit soils (pit, soil_type)",
        dimen=2
    )
    
    # Create pit_id to object mapping for time window constraints
    pit_to_object = {}
    for _, row in layers_df.iterrows():
        pit_to_object[row['pit_id']] = row['object']
    
    # Store the mapping in the model for later use
    model.pit_to_object = pit_to_object
    
    # Extract unique objects for time window processing
    cut_objects = layers_df[layers_df['pit_type'] == 'cut']['object'].unique().tolist()
    fill_objects = layers_df[layers_df['pit_type'] == 'fill']['object'].unique().tolist()
    all_objects = list(set(cut_objects + fill_objects))
    
    # Define a set of all objects for constraints
    model.OBJECTS = pyo.Set(initialize=all_objects, doc="All objects")
    
    # Process time windows at object level
    if df_planning is not None:
        object_time_windows, object_end_periods, month_labels = process_time_windows(
            df_planning, time_periods, all_objects
        )
        print("Time windows processed from planning data at object level")
    else:
        print("No planning data provided.")
    
    # Store time window data as model attributes for later use
    model.object_time_windows = object_time_windows
    model.object_end_periods = object_end_periods
    model.month_labels = month_labels
    
    # Print time windows for a few objects to verify logic
    print("\nSample time windows by object:")
    for obj in all_objects[:min(5, len(all_objects))]:
        print(f"  Object {obj}: {object_time_windows.get(obj, 'All periods')}")
    
    print("\nSample pit to object mapping:")
    sample_pits = (cut_pits + fill_pits)[:min(5, len(cut_pits) + len(fill_pits))]
    for pit in sample_pits:
        obj = pit_to_object.get(pit, "Unknown")
        windows = object_time_windows.get(obj, 'All periods')
        print(f"  Pit {pit} → Object {obj}: {windows}")

def _initialize_model_parameters(model, layers_df, costs_df, soil_compatibility):

    # Parameters - volumes and soil applicability types
    def volume_init(model, pit, soil_type):
        rows = layers_df[(layers_df['pit_id'] == pit) & (layers_df['soil_type'] == soil_type)]
        if rows.empty:
            return 0
        return rows['volume'].sum()
    
    model.VOLUME = pyo.Param(
        model.CUT_LAYERS | model.FILL_LAYERS,
        initialize=volume_init,
        doc="Volume of soil in each pit-soil combination"
    )
    
    def applicability_init(model, pit, soil):
        rows = layers_df[(layers_df['pit_id'] == pit) & (layers_df['soil_type'] == soil)]
        if rows.empty:
            return "unknown"
        return rows['applicability'].iloc[0]
    
    model.APPLICABILITY = pyo.Param(
        model.CUT_LAYERS | model.FILL_LAYERS,
        initialize=applicability_init,
        within=pyo.Any,
        doc="Civil engineering applicability for each soil" # Civil engineering class
    )

    def location_init(model, pit, soil):
        rows = layers_df[(layers_df['pit_id'] == pit) & (layers_df['soil_type'] == soil)]
        if rows.empty:
            return "unknown"
        return rows['objecttype'].iloc[0]  # objecttype is'Dijk' or 'Rivier'

    model.LOCATION = pyo.Param(
        model.CUT_LAYERS | model.FILL_LAYERS,
        initialize=location_init,
        within=pyo.Any,
        doc="Location (Dijk/Rivier) for each pit"
    )
    
    # Environmental parameters, not used in this project but left for future use
    
    # if 'bank_side' in layers_df.columns:
    #     def bank_side_init(model, pit, soil):
    #         rows = layers_df[(layers_df['pit_id'] == pit) & (layers_df['soil_type'] == soil)]
    #         if rows.empty or rows['bank_side'].isna().all():
    #             return None
    #         return rows['bank_side'].iloc[0]
        
    #     model.BANK_SIDE = pyo.Param(
    #         model.CUT_LAYERS | model.FILL_LAYERS,
    #         initialize=bank_side_init,
    #         within=pyo.Any,
    #         doc="Bank side (WB/LB) for each soil"
    #     )
    
    # if 'env_class' in layers_df.columns:
    #     def env_class_init(model, pit, soil):
    #         rows = layers_df[(layers_df['pit_id'] == pit) & (layers_df['soil_type'] == soil)]
    #         if rows.empty or rows['env_class'].isna().all():
    #             return None
    #         return rows['env_class'].iloc[0]
        
    #     model.ENV_CLASS = pyo.Param(
    #         model.CUT_LAYERS | model.FILL_LAYERS,
    #         initialize=env_class_init,
    #         within=pyo.Any,
    #         doc="Environmental class for each soil"
    #     )
    
    # Transportation costs
    def transport_cost_init(model, i, j):
        row = costs_df[(costs_df['source'] == i) & (costs_df['destination'] == j)]
        if row.empty:
            # Use a high cost as fallback, assuming only allowed routes are properly defined 
            return 15
        return float(row['cost'].iloc[0])
    
    model.TRANSPORT_COST = pyo.Param(
        model.CUT_PITS, model.FILL_PITS,
        initialize=transport_cost_init,
        doc="Transportation cost from cut pit to fill pit"
    )
    
    # Higher costs for disposal and borrow operations
    model.DISPOSAL_COST = pyo.Param(initialize=15, doc="Cost to dispose of one unit of soil") 
    model.BORROW_COST = pyo.Param(initialize=150, doc="Cost to borrow one unit of soil") # Estimate based on cost of purchasing and transporting soil
    model.COST_TO_STORAGE = pyo.Param(initialize=3.5, doc="Cost to move one unit of soil to storage") # Assuming central storage is about 3.5km away from all cut pits
    model.COST_FROM_STORAGE = pyo.Param(initialize=3.5, doc="Cost to move one unit of soil from storage") # Assuming central storage is about 3.5km away from all fill pits

    # Daily capacity constraint parameter (10000 m³ per day, ~22 working days per month)
    model.DAILY_CAPACITY = pyo.Param(initialize=10000, doc="Maximum daily transportation capacity (m³)") # 3/4 sets of equipment with about 3000 m³ capacity each
    model.WORKING_DAYS_PER_MONTH = pyo.Param(initialize=22, doc="Working days per month")
    model.MONTHLY_CAPACITY = pyo.Param(
        initialize= 10_000 * 22, 
        doc="Maximum monthly transportation capacity (m³)"
    )
def _setup_allowed_movements(model, soil_compatibility, layers_df=None):

    # Pre-process to allow movements based on soil compatibility
    allowable_movements = set()
    
    for (i, k) in model.CUT_LAYERS:
        for (j, l) in model.FILL_LAYERS:
            # Get soil applicability for compatibility check
            cut_app = model.APPLICABILITY[(i, k)]
            fill_app = model.APPLICABILITY[(j, l)]
            
            if hasattr(model, 'LOCATION'):
                cut_loc = model.LOCATION[(i, k)]
                fill_loc = model.LOCATION[(j, l)]
            else:
                print(f"Warning: Cannot determine location for pits, compatibility may be incorrect")
                continue
            
            # Create composite keys for compatibility check
            cut_key = (cut_loc, cut_app)
            fill_key = (fill_loc, fill_app)
            
            # Check if compatible in our compatibility matrix
            if cut_key in soil_compatibility and fill_key in soil_compatibility.get(cut_key, {}):
                allowable_movements.add((i, k, j, l))
                
    print(f"Found {len(allowable_movements)} allowable movements")
    
    # Store the allowable movements in the model
    model.ALLOWED_MOVEMENTS = pyo.Set(
        initialize=allowable_movements,
        dimen=4,
        doc="Allowable movements between cut and fill soils"
    )

    # Add priority parameter to the model
    def priority_init(model, i, k, j, l):
        # Get soil applicability and location
        cut_app = model.APPLICABILITY[(i, k)]
        fill_app = model.APPLICABILITY[(j, l)]
        
        if hasattr(model, 'LOCATION'):
            cut_loc = model.LOCATION[(i, k)]
            fill_loc = model.LOCATION[(j, l)]
        
        # Create composite keys
        cut_key = (cut_loc, cut_app)
        fill_key = (fill_loc, fill_app)
        
        # Get priority from compatibility dictionary
        if cut_key in soil_compatibility and fill_key in soil_compatibility.get(cut_key, {}):
            return soil_compatibility[cut_key][fill_key]
        return 1  # Default priority
        
    model.MOVEMENT_PRIORITY = pyo.Param(
        model.ALLOWED_MOVEMENTS,
        initialize=priority_init,
        doc="Priority value for each soil movement"
    )

    # Compatibility check for storage-to-fill movements by tracking original cut layer info and destination fill layer
    allowable_storage_to_fill_paths = set()
    for (i_orig, k_orig) in model.CUT_LAYERS:
        for s_site in model.STORAGE_SITES: 
            for (j_fill, l_fill) in model.FILL_LAYERS:
                if (i_orig, k_orig, j_fill, l_fill) in model.ALLOWED_MOVEMENTS:
                    allowable_storage_to_fill_paths.add((i_orig, k_orig, s_site, j_fill, l_fill))

    print(f"Found {len(allowable_storage_to_fill_paths)} allowable storage-to-fill movements")

    model.ALLOWED_STORAGE_TO_FILL_PATHS = pyo.Set(
        initialize=allowable_storage_to_fill_paths,
        dimen=5, # (i_orig, k_orig, s_site, j_fill, l_fill)
        doc="Allowable movements from storage (with original cut layer info) to fill soils"
    )

def _initialize_model_variables(model, time_periods):

    # Transportation variables
    model.x = pyo.Var(
        model.ALLOWED_MOVEMENTS, model.T,
        domain=pyo.NonNegativeReals,
        doc="Volume moved from cut soil to fill soil at time t"
    )
    
    # Disposal and borrow variables
    model.x_disposal = pyo.Var(
        model.CUT_LAYERS, model.DISPOSAL_PITS, model.T,
        domain=pyo.NonNegativeReals,
        doc="Volume moved from cut soil to disposal at time t"
    )
    
    model.x_borrow = pyo.Var(
        model.BORROW_PITS, model.FILL_LAYERS, model.T,
        domain=pyo.NonNegativeReals,
        doc="Volume moved from borrow to fill soil at time t"
    )
    
    # Activity variables (binary)
    model.y_cut = pyo.Var(
        model.CUT_LAYERS, model.T,
        domain=pyo.Binary,
        doc="1 if cut soil is excavated at time t"
    )
    
    model.y_fill = pyo.Var(
        model.FILL_LAYERS, model.T,
        domain=pyo.Binary,
        doc="1 if fill soil is filled at time t"
    )

    # Add inventory variables to track available soil in cut pit at each time period
    model.inventory = pyo.Var(
        model.CUT_LAYERS, model.T,
        domain=pyo.NonNegativeReals,
        doc="Inventory of soil at cut pit at end of period t"
    )
    
    # Binary variable to indicate if an object has any activity (cut or fill) in period t
    model.has_activity = pyo.Var(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        domain=pyo.Binary,
        doc="1 if object has any activity in period t"
    )
    
    # Start and end period variables for each object
    model.start_period = pyo.Var(
        model.OBJECTS,
        domain=pyo.NonNegativeIntegers,
        bounds=(1, time_periods),
        doc="Period when activity at an object starts"
    )
    
    model.end_period = pyo.Var(
        model.OBJECTS,
        domain=pyo.NonNegativeIntegers,
        bounds=(1, time_periods),
        doc="Period when activity at an object ends"
    )
    
    # Binary variable to indicate if an object has any cutting/filling activity at all
    model.has_any_activity = pyo.Var(
        model.OBJECTS,
        domain=pyo.Binary,
        doc="1 if object has any activity in any period"
    )
    
    # Binary variable to indicate if an object has had any cut activity by period t
    model.has_had_cut = pyo.Var(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        domain=pyo.Binary,
        doc="1 if object has had any cut activity by period t"
    )
    
    # Binary variables for logical conditions
    model.before_start = pyo.Var(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        domain=pyo.Binary,
        doc="1 if time t is before start_period for object obj"
    )
    
    model.after_end = pyo.Var(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        domain=pyo.Binary,
        doc="1 if time t is after end_period for object obj"
    )

    # Storage flow variables and inventory variables
    model.x_to_storage = pyo.Var(
        model.CUT_LAYERS, model.STORAGE_SITES, model.T,
        domain=pyo.NonNegativeReals,
        doc="Volume moved from cut soil to storage site at time t"
    )

    model.inventory_storage = pyo.Var(
        model.CUT_LAYERS, model.STORAGE_SITES, model.T, # Index by original cut layer
        domain=pyo.NonNegativeReals,
        doc="Inventory at storage s of soil originally from cut_layer (i_orig, k_orig) at end of period t"
    )

    model.x_from_storage = pyo.Var(
        model.ALLOWED_STORAGE_TO_FILL_PATHS, model.T,
        domain=pyo.NonNegativeReals,
        doc="Volume from storage (orig_cut_layer, storage_site) to fill_layer at time t"
    )

    # Makespan variable to track total project duration
    model.makespan = pyo.Var(domain=pyo.NonNegativeIntegers, 
                        bounds=(1, model.T.last()))
    
def _create_objective_function(model):

    def total_cost_expr(model):
        transport_cost = sum(
            model.x[i, k, j, l, t] * model.TRANSPORT_COST[i, j]
            for (i, k, j, l) in model.ALLOWED_MOVEMENTS
            for t in model.T
        )
        disposal_cost = sum(
            model.x_disposal[i, k, 'disposal', t] * model.DISPOSAL_COST
            for (i, k) in model.CUT_LAYERS 
            for t in model.T
        )
        borrow_cost = sum(
            model.x_borrow['borrow', j, l, t] * model.BORROW_COST
            for (j, l) in model.FILL_LAYERS 
            for t in model.T
        )
        cost_to_storage = sum(
            model.x_to_storage[i, k, s, t] * model.COST_TO_STORAGE
            for (i, k) in model.CUT_LAYERS
            for s in model.STORAGE_SITES
            for t in model.T
        )
        cost_from_storage = sum(
            model.x_from_storage[i_orig, k_orig, s_site, j_fill, l_fill, t] * model.COST_FROM_STORAGE
            for (i_orig, k_orig, s_site, j_fill, l_fill) in model.ALLOWED_STORAGE_TO_FILL_PATHS # Sum over allowed paths
            for t in model.T
        )

        return transport_cost + disposal_cost + borrow_cost + cost_to_storage + cost_from_storage

    model.total_cost = pyo.Expression(rule=total_cost_expr)
    
    def obj_rule(model):

        total_cost = model.total_cost 
        
        w_cost = 2.0 # Punish cost more heavily
        w_makespan = 1000000  # Adjusted based on relative magnitude of total cost
        
        return w_cost * total_cost + w_makespan * (model.makespan / model.T.last())
    
    model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

### 2.5 Adding Constraints

In [ ]:
def _add_volume_balance_constraints(model):

    # Volume balance for cut soils (including disposal)
    def cut_volume_rule(model, i, k):
        fill_destinations = [(j, l) for (i2, k2, j, l) in model.ALLOWED_MOVEMENTS if i2 == i and k2 == k]

        return sum(
            model.x[i, k, j, l, t]
            for (j, l) in fill_destinations
            for t in model.T
        ) + sum(
            model.x_disposal[i, k, 'disposal', t]
            for t in model.T
        ) + sum( 
            model.x_to_storage[i, k, s, t]
            for s in model.STORAGE_SITES
            for t in model.T
        ) == model.VOLUME[i, k] # Hard constraint that volume excavated must be exactly equal to volume in pit

    model.cut_volume_constraint = pyo.Constraint(
        model.CUT_LAYERS, rule=cut_volume_rule,
        doc="Volume excavated from cut soil (including disposal) must equal its volume"
    )
    
    def fill_volume_rule(model, j, l): 
        cut_sources = [(i, k) for (i, k, j2, l2) in model.ALLOWED_MOVEMENTS if j2 == j and l2 == l]

        direct_inflow = sum(
            model.x[i, k, j, l, t]
            for (i, k) in cut_sources
            for t in model.T
        )
        borrow_inflow = sum(
            model.x_borrow['borrow', j, l, t]
            for t in model.T
        )
        storage_inflow = sum(
            model.x_from_storage[i, k, s, j, l, t]
            for (i, k, s, j2, l2) in model.ALLOWED_STORAGE_TO_FILL_PATHS  if j2 == j and l2 == l
            for t in model.T
        )
        return direct_inflow + borrow_inflow + storage_inflow == model.VOLUME[j, l] # Hard constraint that volume excavated must be exactly equal to volume in pit

    model.fill_volume_constraint = pyo.Constraint(
        model.FILL_LAYERS, rule=fill_volume_rule,
        doc="Volume filled into fill soil (including borrow and storage) must meet its volume"
    )

def _add_linking_constraints(model):

    # Link variables for movement
    def link_cut_rule(model, i, k, t):
        fill_destinations = [(j, l) for (i2, k2, j, l) in model.ALLOWED_MOVEMENTS if i2 == i and k2 == k]

        outflow = sum(
            model.x[i, k, j, l, t] for (j, l) in fill_destinations
        ) + sum(
            model.x_disposal[i, k, 'disposal', t] 
        ) + sum( 
            model.x_to_storage[i, k, s, t] for s in model.STORAGE_SITES
        )
        
        # Big M value - max volume that could flow
        big_m = model.VOLUME[i, k]
        
        # Flow can only happen if y_cut is 1
        return outflow <= big_m * model.y_cut[i, k, t]
    
    model.link_cut_constraint = pyo.Constraint(
        model.CUT_LAYERS, model.T,
        rule=link_cut_rule,
        doc="Flow from cut soil is only allowed if activity variable is 1"
    )
    
    def link_fill_rule(model, j, l, t): 
        cut_sources = [(i, k) for (i, k, j2, l2) in model.ALLOWED_MOVEMENTS if j2 == j and l2 == l]

        direct_inflow = sum(
            model.x[i_cut, k_cut, j, l, t] for (i_cut, k_cut) in cut_sources
        )
        borrow_inflow = sum(
            model.x_borrow['borrow', j, l, t] 
        )
        storage_inflow = sum(
            model.x_from_storage[i, k, s, j, l, t]
            for (i, k, s, j2, l2) in model.ALLOWED_STORAGE_TO_FILL_PATHS if j2 == j and l2 == l
        )
        
        inflow = direct_inflow + borrow_inflow + storage_inflow
        big_m = model.VOLUME[j, l]
        return inflow <= big_m * model.y_fill[j, l, t]

    model.link_fill_constraint = pyo.Constraint(
        model.FILL_LAYERS, model.T,
        rule=link_fill_rule,
        doc="Flow to fill soil is only allowed if activity variable is 1"
    )

    # Min flow constraint commented out for now, as it is not used in the current model
    
    # def min_combined_flow_rule(model, obj, t):

    #     # Get all cut and fill pits for this object
    #     cut_pits = [i for i in model.CUT_PITS if model.pit_to_object.get(i) == obj]
    #     fill_pits = [j for j in model.FILL_PITS if model.pit_to_object.get(j) == obj]
        
    #     # Skip if no pits for this object
    #     if not cut_pits and not fill_pits:
    #         return pyo.Constraint.Skip
        
    #     # Minimum flow 
    #     min_flow = (0.5 * pyo.value(model.MONTHLY_CAPACITY)) / 26  
        
    #     # Calculate total CUT flow for this object in this period
    #     cut_flow = sum(
    #         model.x[i, k, j, l, t] 
    #         for (i, k) in cut_pits
    #         for (i2, k2, j, l) in model.ALLOWED_MOVEMENTS if i2 == i and k2 == k
    #     ) + sum(
    #         model.x_disposal[i, k, 'disposal', t]
    #         for (i, k) in cut_pits
    #     ) + sum( 
    #         model.x_to_storage[i, k, s, t]
    #         for (i, k) in cut_pits
    #         for s in model.STORAGE_SITES
    #     )
        
    #     # Calculate total FILL flow for this object in this period
    #     fill_flow = sum(
    #         model.x[i, k, j, l, t]
    #         for (j,l) in fill_pits
    #         for (i, k, j2, l2) in model.ALLOWED_MOVEMENTS if j2 == j and l2 == l
    #     ) + sum(
    #         model.x_borrow['borrow', j, l, t]
    #         for (j,l) in fill_pits
    #     ) + sum(
    #         model.x_from_storage[i, k, s, j, l, t]
    #         for (j, l) in fill_pits
    #         for (i, k, s, j2, l2) in model.ALLOWED_STORAGE_TO_FILL_PATHSif if j2 == j and l2 == l 
    #     )
        
    #     # Combined flow must meet minimum when active
    #     # If has_activity = 0, constraint is satisfied automatically due to Big M
    #     M = pyo.value(model.MONTHLY_CAPACITY)  # Big M value
    #     return cut_flow + fill_flow >= min_flow * model.has_activity[obj, t]

    # # Add the constraint to your model
    # model.min_combined_flow_constraint = pyo.Constraint(
    #     [(obj, t) for obj in model.OBJECTS for t in model.T],
    #     rule=min_combined_flow_rule,
    #     doc="Minimum combined flow for active objects"
    # )

def _add_time_window_constraints(model):

    # Time window constraints,  activities can only happen during allowed time periods
    def cut_time_window_rule(model, i, k, t):
        # Get the object_id for this pit
        object_id = model.pit_to_object.get(i)
        if object_id is None:
            return pyo.Constraint.Skip
            
        # Check if this time period is allowed for this object
        if object_id in model.object_time_windows and t not in model.object_time_windows[object_id]:
            # If this time period is not allowed, prevent any activity
            return model.y_cut[i, k, t] == 0
        return pyo.Constraint.Skip
    
    model.cut_time_window_constraint = pyo.Constraint(
        model.CUT_LAYERS, model.T,
        rule=cut_time_window_rule,
        doc="Restrict cut activities to their allowed time windows"
    )
    
    def fill_time_window_rule(model, j, l, t):
        # Get the object_id for this pit
        object_id = model.pit_to_object.get(j)
        if object_id is None:
            return pyo.Constraint.Skip
            
        # Check if this time period is allowed for this object
        if object_id in model.object_time_windows and t not in model.object_time_windows[object_id]:
            # If this time period is not allowed, prevent any activity
            return model.y_fill[j, l, t] == 0
        return pyo.Constraint.Skip
    
    model.fill_time_window_constraint = pyo.Constraint(
        model.FILL_LAYERS, model.T,
        rule=fill_time_window_rule,
        doc="Restrict fill activities to their allowed time windows"
    )
    
    # End time constraint, all fill operations must be completed by the end period
    def fill_completion_rule(model, j, l):
        # Get the object_id for this pit
        object_id = model.pit_to_object.get(j)
        if object_id is None or object_id not in model.object_end_periods:
            return pyo.Constraint.Skip
        
        end_period = model.object_end_periods[object_id]
        
        # Get all allowed cut sources for this fill soil
        cut_sources = [(i, k) for (i, k, j2, l2) in model.ALLOWED_MOVEMENTS 
                    if j2 == j and l2 == l]
        
        # Check if there are any periods after the end period
        later_periods = [t for t in model.T if t > end_period]
        
        # If no later periods or no cut sources, skip the constraint
        if not later_periods or not cut_sources:
            return pyo.Constraint.Skip
        
        # Create expressions for flows after end period
        flow_vars = [model.x[i, k, j, l, t] for (i, k) in cut_sources for t in later_periods]
        borrow_vars = [model.x_borrow['borrow', j, l, t] for t in later_periods]
        
        # Combine all variables that should be zero
        all_vars = flow_vars + borrow_vars
        
        # If no variables to constrain, return Feasible
        if not all_vars:
            return pyo.Constraint.Feasible
        
        # Otherwise create a proper Pyomo expression
        return sum(all_vars) == 0
    
    model.fill_completion_constraint = pyo.Constraint(
        model.FILL_LAYERS,
        rule=fill_completion_rule,
        doc="Ensure all fill operations are completed by the end of their time window"
    )

def _add_capacity_constraints(model):
   
    def object_capacity_constraint_rule(model, t):
        # Only count transportation between DIFFERENT objects
        normal_transport = sum(
            model.x[i, k, j, l, t] for (i, k, j, l) in model.ALLOWED_MOVEMENTS 
            if model.pit_to_object.get(i) != model.pit_to_object.get(j)
        )
        
        # Total disposal and borrow
        disposal = sum(model.x_disposal[i, k, 'disposal', t] for (i, k) in model.CUT_LAYERS)
        borrow = sum(model.x_borrow['borrow', j, l, t] for (j, l) in model.FILL_LAYERS)

        # Add flows to/from storage
        to_storage_transport = sum(
            model.x_to_storage[i, k, s, t]
            for (i,k) in model.CUT_LAYERS
            for s in model.STORAGE_SITES
        )

        from_storage_transport = sum(
            model.x_from_storage[i, k, s, j, l, t]
            for (i, k, s, j, l) in model.ALLOWED_STORAGE_TO_FILL_PATHS
        )

        # Total flow must be within monthly capacity
        return normal_transport + disposal + borrow + to_storage_transport + from_storage_transport <= model.MONTHLY_CAPACITY
    
    model.capacity_constraint = pyo.Constraint(
        model.T,
        rule=object_capacity_constraint_rule,
        doc="Limit total soil movement to monthly capacity"
    )

    # Restict borrow operations to be zero for fill layers to maximise reuse of existing soils
    model.no_borrow_allowed = pyo.Constraint(
        model.FILL_LAYERS,
        rule=lambda model, j, l: sum(
            model.x_borrow['borrow', j, l, t] for t in model.T
        ) == 0,
        doc="No borrow allowed for any fill layer"
    )

def _add_inventory_constraints(model):

    # Inventory balance constraints
    def inventory_balance_rule(model, i, k, t):
        # Calculate all outflows from this cut soil in this period
        outflow = sum(
            model.x[i, k, j, l, t] for (j, l) in model.FILL_LAYERS 
            if (i, k, j, l) in model.ALLOWED_MOVEMENTS
        ) + sum(
            model.x_disposal[i, k, 'disposal', t] 
        ) + sum(
             model.x_to_storage[i, k, s, t] for s in model.STORAGE_SITES
        )
        
        if t == 1:
            # Initial period - starting inventory is the volume in the pit
            return model.inventory[i, k, t] == model.VOLUME[i, k]
        else:
            # Other periods - inventory carries over
            return model.inventory[i, k, t] == (model.inventory[i, k, t-1] - outflow)

    model.inventory_balance = pyo.Constraint(
        model.CUT_LAYERS, model.T,
        rule=inventory_balance_rule,
        doc="Track soil inventory balance over time")
    
    # Cannot ship more than available inventory
    def inventory_limit_rule(model, i, k, t):
        # Calculate all outflows from this cut soil in this period
        outflow = sum(
            model.x[i, k, j, l, t] for (j, l) in model.FILL_LAYERS 
            if (i, k, j, l) in model.ALLOWED_MOVEMENTS
        ) + sum(
            model.x_disposal[i, k, 'disposal', t] 
        ) + sum(
            model.x_to_storage[i, k, s, t] for s in model.STORAGE_SITES
        )
        
        # If t > 1, we can use the previous inventory, otherwise we use the initial volume, only cut if allowed
        return outflow <= model.inventory[i, k, t - 1] if t > 1 else model.VOLUME[i, k]

    model.inventory_limit = pyo.Constraint(
        model.CUT_LAYERS, model.T,
        rule=inventory_limit_rule,
        doc="Cannot ship more than available inventory"
    )

    def storage_inventory_balance_rule(model, i, k, s, t):
        # Inflow to storage for this specific cut layer (i, k) to this storage site s
        inflow_to_storage = model.x_to_storage[i, k, s, t]

        # Outflow from storage of soil originally from (i, k) from this storage site s
        outflow_from_storage = sum(
            model.x_from_storage[i, k, s, j, l, t]
            for (i2, k2, s, j, l) in model.ALLOWED_STORAGE_TO_FILL_PATHS
            if i2 == i and k2 == k
        )

        if t == model.T.first(): # First time period
            return model.inventory_storage[i, k, s, t] == \
                inflow_to_storage - outflow_from_storage
        else: # Subsequent time periods
            return model.inventory_storage[i, k, s, t] == \
                model.inventory_storage[i, k, s, t - 1] + \
                inflow_to_storage - outflow_from_storage

    model.storage_inventory_balance = pyo.Constraint(
        model.CUT_LAYERS, model.STORAGE_SITES, model.T,
        rule=storage_inventory_balance_rule,
        doc="Track inventory balance in storage for each original cut soil layer"
    )

    def storage_inventory_limit_rule(model, i, k, s, t):
        outflow_from_storage = sum(
            model.x_from_storage[i, k, s, j, l, t]
            for (i2, k2, s, j, l) in model.ALLOWED_STORAGE_TO_FILL_PATHS
            if i2 == i and k2 == k
        )
        inflow_to_storage = model.x_to_storage[i, k, s, t]

        if t == model.T.first():
            return outflow_from_storage <= inflow_to_storage
        else:
            return outflow_from_storage <= model.inventory_storage[i, k, s, t - 1] + inflow_to_storage

    model.storage_inventory_limit = pyo.Constraint(
        model.CUT_LAYERS, model.STORAGE_SITES, model.T,
        rule=storage_inventory_limit_rule,
        doc="Cannot ship more from storage than available from that origin"
    )

def _add_priority_constraints(model, epsilon=0.001):

    # Identify priority levels and group movements
    priority_scores = sorted(list(set(pyo.value(model.MOVEMENT_PRIORITY[i, k, j, l])
                                   for i, k, j, l in model.ALLOWED_MOVEMENTS)))

    priority_movements_dict = {level: [] for level in priority_scores}
    for i, k, j, l in model.ALLOWED_MOVEMENTS:
        priority_movements_dict[pyo.value(model.MOVEMENT_PRIORITY[i, k, j, l])].append((i, k, j, l))

    # Define X_actual_p and V_potential_p for each priority level
    model.X_actual_p = pyo.Param(priority_scores, mutable=True, within=pyo.Any) # Will store expressions
    model.V_potential_p = pyo.Param(priority_scores, initialize=0.0, mutable=True)

    for p_score in priority_scores:
        actual_flow_expr = 0
        potential_flow_sum = 0
        
        if not priority_movements_dict[p_score]:
            model.X_actual_p[p_score] = 0.0
            model.V_potential_p[p_score] = 0.0
            continue

        for i, k, j, l in priority_movements_dict[p_score]:
            # Direct flow component
            direct_flow_for_path = sum(model.x[i, k, j, l, t] for t in model.T)
            actual_flow_expr += direct_flow_for_path

            if hasattr(model, 'x_from_storage'): #
                 if (i,k,j,l) in [(st_i,st_k,st_j,st_l) for st_i,st_k,_,st_j,st_l,_ in getattr(model, 'ALLOWED_STORAGE_TO_FILL_PATHS', [])]:
                    storage_flow_for_path = sum(model.x_from_storage[i, k, s, j, l, t]
                                                for s in model.S_SITES 
                                                for t in model.T
                                                if (i,k,s,j,l,t) in model.x_from_storage) # Check index validity
                    actual_flow_expr += storage_flow_for_path

            supply_ik = model.VOLUME[i,k] if (i,k) in model.VOLUME else 0
            demand_jl = model.VOLUME[j,l] if (j,l) in model.VOLUME else 0
            potential_flow_sum += min(pyo.value(supply_ik), pyo.value(demand_jl))
        
        model.X_actual_p[p_score] = actual_flow_expr
        model.V_potential_p[p_score] = potential_flow_sum

    # Define binary variables delta_p

    if len(priority_scores) > 1:
        model.delta_priority_unlocked = pyo.Var(
            [priority_scores[i] for i in range(len(priority_scores)-1)],
            domain=pyo.Binary
        )

    # Define M_switch (Big-M)
    M_switch = sum(pyo.value(model.V_potential_p[p_score]) for p_score in priority_scores)
    if M_switch == 0: # Avoid division by zero or tiny M if no potential flow
        M_switch = 1.0e6 # A default large number

    # Add constraints
    model.priority_unlock_constraints = pyo.ConstraintList()
    model.priority_flow_gate_constraints = pyo.ConstraintList()

    X_CUM_actual = 0
    V_CUM_potential = 0

    for i in range(len(priority_scores)):
        p_score = priority_scores[i]
        
        # Add current level's actual and potential to cumulative sums
        
        if isinstance(model.X_actual_p[p_score], (int, float)) and model.X_actual_p[p_score] == 0.0:
            pass # No actual flow expression to add for this priority
        else:
            X_CUM_actual += model.X_actual_p[p_score]
        
        V_CUM_potential += pyo.value(model.V_potential_p[p_score])

        if i < len(priority_scores) - 1: # For all but the last priority level
            # Constraint (1a): Unlock condition
            # V_CUM_potential(p) - X_CUM_actual(p) <= M_switch * (1 - delta(p)) + epsilon
            delta_var = model.delta_priority_unlocked[p_score]
            lhs_unlock = V_CUM_potential - X_CUM_actual
            rhs_unlock = M_switch * (1 - delta_var) + epsilon
            model.priority_unlock_constraints.add(lhs_unlock <= rhs_unlock)

            # Constraint (1b) Flow gate for NEXT priority level
            # X_actual(p+1) <= V_potential(p+1) * delta(p)
            next_p_score = priority_scores[i+1]
            
            if isinstance(model.X_actual_p[next_p_score], (int, float)) and model.X_actual_p[next_p_score] == 0.0:
                # If X_actual for next priority is 0 (e.g. no movements), constraint is 0 <= V_pot * delta
                if pyo.value(model.V_potential_p[next_p_score]) > 0:
                     model.priority_flow_gate_constraints.add(
                        0.0 <= pyo.value(model.V_potential_p[next_p_score]) * delta_var
                    )
            else:
                model.priority_flow_gate_constraints.add(
                    model.X_actual_p[next_p_score] <= pyo.value(model.V_potential_p[next_p_score]) * delta_var
                )

def _add_logic_constraints(model):

    # Link has_activity to y_cut and y_fill 
    def link_activity_rule1(model, obj, t):
        
        # Find all cut and fill pits for this object
        cut_pits_for_obj = [i for i in model.CUT_PITS if model.pit_to_object.get(i) == obj]
        fill_pits_for_obj = [j for j in model.FILL_PITS if model.pit_to_object.get(j) == obj]
        
        # If no pits for this object, skip
        if not cut_pits_for_obj and not fill_pits_for_obj:
            return pyo.Constraint.Skip
        
        # Calculate total activity
        cut_activity = sum(
            model.y_cut[i, k, t]
            for i in cut_pits_for_obj
            for k in [k for (i2, k) in model.CUT_LAYERS if i2 == i]
        )
        
        fill_activity = sum(
            model.y_fill[j, l, t]
            for j in fill_pits_for_obj
            for l in [l for (j2, l) in model.FILL_LAYERS if j2 == j]
        )
        
        M = max(len(cut_pits_for_obj) * len(model.T), len(fill_pits_for_obj) * len(model.T), 1)

        return cut_activity + fill_activity <= M * model.has_activity[obj, t]

    model.link_activity_constraint1 = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        rule=link_activity_rule1,
        doc="Link has_activity to cutting and filling variables"
    )
    
    # 10. Link has_activity to y_cut and y_fill
    def link_activity_rule2(model, obj, t):
        
        # Find all cut and fill pits for this object
        cut_pits_for_obj = [i for i in model.CUT_PITS if model.pit_to_object.get(i) == obj]
        fill_pits_for_obj = [j for j in model.FILL_PITS if model.pit_to_object.get(j) == obj]
        
        # If no pits for this object, skip
        if not cut_pits_for_obj and not fill_pits_for_obj:
            return pyo.Constraint.Skip
        
        # Calculate total activity
        cut_activity = sum(
            model.y_cut[i, k, t]
            for i in cut_pits_for_obj
            for k in [k for (i2, k) in model.CUT_LAYERS if i2 == i]
        )
        
        fill_activity = sum(
            model.y_fill[j, l, t]
            for j in fill_pits_for_obj
            for l in [l for (j2, l) in model.FILL_LAYERS if j2 == j]
        )
        
        M = max(len(cut_pits_for_obj) * len(model.T), len(fill_pits_for_obj) * len(model.T), 1)
        
        # If has_activity is 1, then there must be at least some activity
        if cut_pits_for_obj or fill_pits_for_obj:
            return model.has_activity[obj, t] <= cut_activity + fill_activity + \
                (1 - model.has_activity[obj, t]) * M
        else:
            return pyo.Constraint.Skip

    model.link_activity_constraint2 = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        rule=link_activity_rule2,
        doc="Link has_activity to cutting and filling variables"
    )

    # Link has_any_activity to has_activity
    def link_any_activity_rule1(model, obj):
        
        # has_any_activity must be 1 if there's activity in any period
        return sum(model.has_activity[obj, t] for t in model.T) <= len(model.T) * model.has_any_activity[obj]
    
    model.link_any_activity_constraint1 = pyo.Constraint(
        model.OBJECTS,
        rule=link_any_activity_rule1,
        doc="Link has_any_activity to has_activity"
    )
    
    # Link has_any_activity to has_activity 
    def link_any_activity_rule2(model, obj):
        
        # If has_any_activity is 1, there must be at least one period with activity
        return model.has_any_activity[obj] <= sum(model.has_activity[obj, t] for t in model.T)
    
    model.link_any_activity_constraint2 = pyo.Constraint(
        model.OBJECTS,
        rule=link_any_activity_rule2,
        doc="Link has_any_activity to has_activity"
    )

    # Add constraints to link binary indicators with time relationships
    
    # Link before_start indicator with start_period
    def link_before_start_rule1(model, obj, t):

        # Constraint 1: If before_start is 0, then t >= start_period
        M = len(model.T) + 1
        return t + M * model.before_start[obj, t] >= model.start_period[obj]
    
    def link_before_start_rule2(model, obj, t):

        # Constraint 2: If before_start is 1, then t < start_period
        M = len(model.T) + 1
        return t + 1 <= model.start_period[obj] + M * (1 - model.before_start[obj, t])
    
    model.link_before_start_constraint1 = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        rule=link_before_start_rule1,
        doc="Link before_start variable to start_period"
    )
    
    model.link_before_start_constraint2 = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        rule=link_before_start_rule2,
        doc="Link before_start variable to start_period"
    )
    
    # Link after_end indicator with end_period
    def link_after_end_rule1(model, obj, t):

        # Constraint 1: If after_end is 0, then t <= end_period
        M = len(model.T) + 1
        return t <= model.end_period[obj] + M * model.after_end[obj, t]
    
    def link_after_end_rule2(model, obj, t):

        # Constraint 2: If after_end is 1, then t > end_period
        M = len(model.T) + 1
        return t >= model.end_period[obj] + 1 - M * (1 - model.after_end[obj, t])
    
    model.link_after_end_constraint1 = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        rule=link_after_end_rule1,
        doc="Link after_end variable to end_period - part 1"
    )
    
    model.link_after_end_constraint2 = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        rule=link_after_end_rule2,
        doc="Link after_end variable to end_period"
    )
    
    # Define start_period for each object
    def start_period_rule1(model, obj):
        
        M = len(model.T) + 1  # Big M value
        
        # If has_any_activity is 0, start_period can be any value (up to time_periods)
        return model.start_period[obj] <= len(model.T) + M * (1 - model.has_any_activity[obj])
    
    model.start_period_constraint1 = pyo.Constraint(
        model.OBJECTS,
        rule=start_period_rule1,
        doc="Define start_period - part 1"
    )
    
    # Define start_period for each object - For each period with activity, start_period <= t
    def start_period_rule2(model, obj, t):
        M = len(model.T) + 1
        return model.start_period[obj] <= t + M * (1 - model.has_activity[obj, t])
    
    model.start_period_constraint2 = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        rule=start_period_rule2,
        doc="Define start_period - part 2"
    )
    
    # Define start_period for each object - If t < start_period, has_activity must be 0
    def start_period_rule3(model, obj, t):
        """If t < start_period, has_activity must be 0"""
        M = len(model.T) + 1
        # Use before_start indicator instead of direct comparison
        return model.has_activity[obj, t] <= 1 - model.before_start[obj, t] + M * (1 - model.has_any_activity[obj])
    
    model.start_period_constraint3 = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        rule=start_period_rule3,
        doc="Define start_period - part 3"
    )
    
    # Define end_period for each object
    def end_period_rule1(model, obj):
        """Define end_period as the last period with activity - part 1"""
        
        M = len(model.T) + 1  # Big M value
        
        # If has_any_activity is 0, end_period can be any value (as low as 1)
        return model.end_period[obj] >= 1 - M * (1 - model.has_any_activity[obj])
    
    model.end_period_constraint1 = pyo.Constraint(
        model.OBJECTS,
        rule=end_period_rule1,
        doc="Define end_period - part 1"
    )
    
    # Define end_period for each object - For each period with activity, end_period >= t
    def end_period_rule2(model, obj, t):
        return model.end_period[obj] >= t * model.has_activity[obj, t]
    
    model.end_period_constraint2 = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        rule=end_period_rule2,
        doc="Define end_period - part 2"
    )
    
    # Define end_period for each object - If t > end_period, has_activity must be 0
    def end_period_rule3(model, obj, t):
        M = len(model.T) + 1
        # Use after_end indicator instead of direct comparison
        return model.has_activity[obj, t] <= 1 - model.after_end[obj, t] + M * (1 - model.has_any_activity[obj])
    
    model.end_period_constraint3 = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T],
        rule=end_period_rule3,
        doc="Define end_period - part 3"
    )
    
    # Ensure start_period <= end_period
    def period_ordering_rule(model, obj):
        """Ensure start_period <= end_period"""
        return model.start_period[obj] <= model.end_period[obj]
    
    model.period_ordering_constraint = pyo.Constraint(
        model.OBJECTS,
        rule=period_ordering_rule,
        doc="Ensure start_period <= end_period"
    )

    def before_start_activity_rule(model, obj, t):
        if obj == 'nan':
            return pyo.Constraint.Skip
        
        # If t is before start_period (before_start=1), then has_activity must be 0
        return model.has_activity[obj, t] <= 1 - model.before_start[obj, t]

    model.before_start_activity_constraint = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T if obj != 'nan'],
        rule=before_start_activity_rule,
        doc="Enforce that activity can't happen before start_period"
    )

    # Activity can't happen after end_period
    def after_end_activity_rule(model, obj, t):

        if obj == 'nan':
            return pyo.Constraint.Skip
        
        # If t is after end_period (after_end=1), then has_activity must be 0
        return model.has_activity[obj, t] <= 1 - model.after_end[obj, t]

    model.after_end_activity_constraint = pyo.Constraint(
        [(obj, t) for obj in model.OBJECTS for t in model.T if obj != 'nan'],
        rule=after_end_activity_rule,
        doc="Enforce that activity can't happen after end_period"
    )

    M = len(model.T)

    def direct_span_constraint_lower_rule(model, obj):

        # Enforce a lower bound on the number of active periods.
        # For an active object (has_any_activity[obj] == 1), this forces:
        #     sum(has_activity[obj, t] for t in T) >= end_period[obj] - start_period[obj] + 1.
        # For an inactive object (has_any_activity[obj] == 0), the right-hand side is reduced by M.
        return sum(model.has_activity[obj, t] for t in model.T) >= model.end_period[obj] - model.start_period[obj] + 1 - (1 - model.has_any_activity[obj]) * M

    def direct_span_constraint_upper_rule(model, obj):

        # Enforce an upper bound on the number of active periods.
        # For an active object, it forces:
        #     sum(has_activity[obj,t] for t in T) <= end_period[obj] - start_period[obj] + 1.
        # For an inactive object, the right-hand side is increased by M (relaxing the constraint).
        return sum(model.has_activity[obj, t] for t in model.T) <= model.end_period[obj] - model.start_period[obj] + 1 + (1 - model.has_any_activity[obj]) * M

    model.direct_span_constraint_lower = pyo.Constraint(
        model.OBJECTS,
        rule=direct_span_constraint_lower_rule,
        doc="Lower bound for direct span constraint"
    )

    model.direct_span_constraint_upper = pyo.Constraint(
        model.OBJECTS,
        rule=direct_span_constraint_upper_rule,
        doc="Upper bound for direct span constraint"
    )

def add_integrated_cut_fill_constraints(model):
    
    # We have to create new binary variables to track the cutting and filling activities
    
    # Create variables to track if all cutting is complete for each pit
    model.all_cut_complete = pyo.Var(
        [(pit_id, t) for pit_id in model.CUT_PITS.intersection(model.FILL_PITS) for t in model.T],
        domain=pyo.Binary,
        doc="1 if all cutting is complete for this pit by period t"
    )
    
    # Create variables to track if any filling has started for each pit
    model.any_fill_started = pyo.Var(
        [(pit_id, t) for pit_id in model.CUT_PITS.intersection(model.FILL_PITS) for t in model.T],
        domain=pyo.Binary,
        doc="1 if any filling has started for this pit by period t"
    )

    # Create binary variables to indicate if each soil has been cut by period t
    model.soil_cut_by_period = pyo.Var(
        [(pit_id, k, t) for pit_id in model.CUT_PITS.intersection(model.FILL_PITS) 
         for k in pit_cut_soils.get(pit_id, []) for t in model.T],
        domain=pyo.Binary,
        doc="1 if soil k at pit_id has been cut by period t"
    )
    
    # Get cut soils for each pit
    pit_cut_soils = {}
    for (i, k) in model.CUT_LAYERS:
        if i not in pit_cut_soils:
            pit_cut_soils[i] = []
        pit_cut_soils[i].append(k)
    
    # Get fill soils for each pit
    pit_fill_soils = {}
    for (j, l) in model.FILL_LAYERS:
        if j not in pit_fill_soils:
            pit_fill_soils[j] = []
        pit_fill_soils[j].append(l)
    
    # Link soil_cut_by_period to actual cutting activities
    def link_soil_cut_rule(model, pit_id, k, t):
        # Skip if invalid combination
        if pit_id not in pit_cut_soils or k not in pit_cut_soils[pit_id]:
            return pyo.Constraint.Skip
        
        # Sum of cut activity for this soil up to period t
        soil_cut = sum(model.y_cut[pit_id, k, tau] for tau in range(1, t+1))
        
        # soil_cut_by_period must be 1 if there was any cut activity
        return model.soil_cut_by_period[pit_id, k, t] >= soil_cut / (t + 0.001)
    
    model.link_soil_cut_constraint = pyo.Constraint(
        [(pit_id, k, t) for pit_id in model.CUT_PITS.intersection(model.FILL_PITS) 
         for k in pit_cut_soils.get(pit_id, []) for t in model.T],
        rule=link_soil_cut_rule,
        doc="Link soil_cut_by_period to actual cutting activities"
    )
    
    # Define all_cut_complete based on all soils being cut
    def track_cut_progress_rule(model, pit_id, t):

        # Skip if pit doesn't have both cut and fill operations
        if pit_id not in pit_cut_soils or pit_id not in pit_fill_soils:
            return pyo.Constraint.Skip
        
        # If no cut soils, consider cutting complete
        if not pit_cut_soils[pit_id]:
            return model.all_cut_complete[pit_id, t] == 1
            
        # all_cut_complete can only be 1 if all soils have been cut
        total_soils = len(pit_cut_soils[pit_id])
        sum_cut_soils = sum(model.soil_cut_by_period[pit_id, k, t] for k in pit_cut_soils[pit_id])
        
        # Force all_cut_complete to be 0 if any soil hasn't been cut yet
        return sum_cut_soils >= total_soils - (1 - model.all_cut_complete[pit_id, t]) * total_soils
    
    model.track_cut_progress_constraint = pyo.Constraint(
        [(pit_id, t) for pit_id in model.CUT_PITS.intersection(model.FILL_PITS) for t in model.T],
        rule=track_cut_progress_rule,
        doc="Track if all cutting is complete by period t"
    )
    
    # Define fill activity tracking constraint
    def track_fill_activity_rule(model, pit_id, t):

        # Skip if pit doesn't have both cut and fill operations
        if pit_id not in pit_cut_soils or pit_id not in pit_fill_soils:
            return pyo.Constraint.Skip
        
        # Sum of all fill activity up to period t
        fill_activity = sum(
            model.y_fill[pit_id, l, tau] 
            for l in pit_fill_soils[pit_id]
            for tau in range(1, t+1)
        )
        
        # Big M - the number of possible fill decisions up to period t
        big_m = len(pit_fill_soils[pit_id]) * t
        
        # any_fill_started must be 1 if there was any fill activity, and can be
        # either 0 or 1 if there was no fill activity
        return fill_activity <= big_m * model.any_fill_started[pit_id, t]
    
    model.track_fill_activity_constraint = pyo.Constraint(
        [(pit_id, t) for pit_id in model.CUT_PITS.intersection(model.FILL_PITS) for t in model.T],
        rule=track_fill_activity_rule,
        doc="Track if any filling has started by period t"
    )
    
    # Add a second constraint to ensure any_fill_started is 0 if no filling
    def track_fill_activity_rule2(model, pit_id, t):

        # Skip if pit doesn't have both cut and fill operations
        if pit_id not in pit_cut_soils or pit_id not in pit_fill_soils:
            return pyo.Constraint.Skip
        
        # Sum of all fill activity up to period t
        fill_activity = sum(
            model.y_fill[pit_id, l, tau] 
            for l in pit_fill_soils[pit_id]
            for tau in range(1, t+1)
        )
        
        # Force any_fill_started to be 0 if there's no filling
        return model.any_fill_started[pit_id, t] <= fill_activity
    
    model.track_fill_activity_constraint2 = pyo.Constraint(
        [(pit_id, t) for pit_id in model.CUT_PITS.intersection(model.FILL_PITS) for t in model.T],
        rule=track_fill_activity_rule2,
        doc="Force any_fill_started to be 0 if no filling has occurred"
    )
    
    # The core cut-before-fill constraint
    def cut_before_fill_rule(model, pit_id, t):

        # Skip if pit doesn't have both cut and fill operations
        if pit_id not in pit_cut_soils or pit_id not in pit_fill_soils:
            return pyo.Constraint.Skip
        
        # Can only fill in period t if all cutting was completed by period t-1
        if t > 1:
            # Sum of all fill activity in this period
            fill_activity_t = sum(model.y_fill[pit_id, l, t] for l in pit_fill_soils[pit_id])
            
            # If all cutting wasn't complete by previous period, no filling can happen
            M = len(pit_fill_soils[pit_id])  # Maximum possible fill activity in one period
            return fill_activity_t <= M * model.all_cut_complete[pit_id, t-1]
        else:
            # For period 1, can only fill if no cutting is needed (e.g., pit has no cut soils)
            # or if all cutting happens in period 1
            fill_activity_1 = sum(model.y_fill[pit_id, l, 1] for l in pit_fill_soils[pit_id])
            M = len(pit_fill_soils[pit_id])
            return fill_activity_1 <= M * model.all_cut_complete[pit_id, 1]
    
    model.cut_before_fill_constraint = pyo.Constraint(
        [(pit_id, t) for pit_id in model.CUT_PITS.intersection(model.FILL_PITS) for t in model.T],
        rule=cut_before_fill_rule,
        doc="Ensure all cutting is complete before any filling starts"
    )
    
    # Prevent further cutting after filling has started
    def no_cut_after_fill_rule(model, pit_id, t):

        # Skip if pit doesn't have both cut and fill operations
        if pit_id not in pit_cut_soils or pit_id not in pit_fill_soils:
            return pyo.Constraint.Skip
        
        # No cutting in period t if filling happened by period t-1
        if t > 1:
            # Sum of all cut activity in this period
            cut_activity_t = sum(model.y_cut[pit_id, k, t] for k in pit_cut_soils[pit_id])
            
            # If filling started by previous period, no cutting can happen
            M = len(pit_cut_soils[pit_id])  # Maximum possible cut activity in one period
            return cut_activity_t <= M * (1 - model.any_fill_started[pit_id, t-1])
        else:
            return pyo.Constraint.Skip
    
    model.no_cut_after_fill_constraint = pyo.Constraint(
        [(pit_id, t) for pit_id in model.CUT_PITS.intersection(model.FILL_PITS) for t in model.T],
        rule=no_cut_after_fill_rule,
        doc="Prevent cutting after filling has started"
    )

    # Add constraint: makespan must be >= end_period for any object
    def makespan_constraint_rule(model, obj):
        return model.makespan >= model.end_period[obj]
    
    model.makespan_constraint = pyo.Constraint(
        model.OBJECTS,
        rule=makespan_constraint_rule
    )

### 2.5 Main Function (Visualization and Saving Solution Code Excluded for Brevity)

In [ ]:
def main():
    """Main function to run the soil transportation optimization."""
    print("Starting Soil Transportation Optimization with Monthly Time Windows and Environmental Compatibility")

    SOLUTION_FILENAME_PREFIX = "soil_transport_solution"

    # Set this flag to True to solve the model, False to load and visualize existing solution

    SOLVE_AND_SAVE_MODEL = False 

    solution_data = None
    
    # Always load and process initial data for context (pit_coords, base_cut_pits, etc.)
    try:
        data_objects = load_data()
        print("Successfully loaded data from pickle file")
    except Exception as e:
        print(f"Error loading data: {e}")
        return

    layers_df, costs_df, pit_coords, soil_compatibility, cut_pits, fill_pits, df_planning = process_data(data_objects)
    
    print(f"layers_df shape: {layers_df.shape}")
    print(f"costs_df shape: {costs_df.shape}")
    print(f"Number of cut_pits: {len(cut_pits)}")
    print(f"Number of fill_pits: {len(fill_pits)}")
    
    if SOLVE_AND_SAVE_MODEL:
        print("\nData Summary (before solving):")
        print(f"Number of soil types: {len(set(layers_df['soil_type']))}")
        print(f"Number of cost entries: {len(costs_df)}")
        
        total_supply = layers_df[layers_df['pit_type'] == 'cut']['volume'].sum()
        total_demand = abs(layers_df[layers_df['pit_type'] == 'fill']['volume'].sum())
        print(f"Total supply: {total_supply:.2f} m³")
        print(f"Total demand: {total_demand:.2f} m³")
        print(f"Net balance: {total_supply - total_demand:.2f} m³ {'(excess supply)' if total_supply > total_demand else '(supply shortage)'}\n")
        
        time_periods = 72       
        time_limit = 1000 # Increased time limit for Gurobi's MIPGap tolerance
        
        model = create_and_solve_model(
            layers_df, costs_df, soil_compatibility, cut_pits, fill_pits, pit_coords,
            time_periods=time_periods, time_limit=time_limit, df_planning=df_planning
        )
        
        if model:
            print("\nSaving solution...")

            # Code for this function excluded from notebook for brevity
            solution_data = save_solution(model, layers_df, filename_prefix=SOLUTION_FILENAME_PREFIX)
        else:
            print("Model solving failed. Cannot visualize or save.")
            return
    else:
        print(f"\nLoading solution from {SOLUTION_FILENAME_PREFIX}.json..")
        try:
            with open(f"{SOLUTION_FILENAME_PREFIX}.json", 'r') as f:
                solution_data = json.load(f)
            print(f"Successfully loaded solution from {SOLUTION_FILENAME_PREFIX}.json")
        except FileNotFoundError:
            print(f"Solution file {SOLUTION_FILENAME_PREFIX}.json not found. Run with SOLVE_AND_SAVE_MODEL=True first.")
            return
        except Exception as e:
            print(f"Error loading solution data: {e}")
            return

    if solution_data and 'summary' in solution_data:
        print("\nVisualizing solution...")
        # Code for this function excluded from notebook for brevity
        visualize_solution_from_data(solution_data, layers_df, pit_coords, cut_pits, fill_pits, df_planning)
    else:
        print("No solution data available for visualization.")

## 3. Running Optimization Script

In [ ]:
main()